# Use k means cluster on data to get classified labels

## Importing and Read & Preprocess data

In [1]:
import global_resources as gr
import numpy as np
import torch
import kmc_torch.kmc as kmc

In [2]:
data_dir = r"D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Cluster DATA\processed_data_final.csv"
df = gr.read_and_return_pd_df(data_dir)
to_drop = ['证券简称', '行业代码1']
df.drop(to_drop, axis = 1, inplace = True)
df['证券代码'] = df['证券代码'].astype(str).str.zfill(6)
Stock_ID_df = df['证券代码']
df.drop(['证券代码'], axis = 1, inplace = True)
display(Stock_ID_df)
display(df)

Reading files from: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Cluster DATA\processed_data_final.csv


0       000001
1       000002
2       000004
3       000006
4       000007
         ...  
5316    920002
5317    920008
5318    920016
5319    920099
5320    920118
Name: 证券代码, Length: 5321, dtype: object

,营业总收入,现金资产比率,应收类资产比率,固定资产比率,无形资产比率,金融负债比率,少数股东权益占比,市盈率（PE）1,资产负债率,市值A,托宾Q值A,账面市值比A,资产报酬率A,净资产收益率（ROE）A
0,1.294663,-0.777553,-1.270996,-1.297656,-0.692415,-1.477725,-0.185208,-0.330143,2.323068,4.379345,-0.733083,1.421790,-0.375351,0.132565
1,3.324778,-0.706152,-1.208915,-1.205924,-0.608636,-0.188667,1.208350,-0.326349,1.446345,0.858528,-0.829444,1.958758,-0.737222,-0.103471
2,-0.151864,0.133231,2.573513,-1.277036,-0.136708,-1.085169,-0.257693,-0.370844,0.840408,-0.068671,3.617179,-2.012428,-2.865983,-0.471754
3,-0.076711,-0.165453,-1.251412,-1.287878,-0.710005,0.400996,0.012098,-0.370844,1.215474,-0.053009,-0.664129,1.108182,-1.008012,-0.196261
4,-0.150626,1.671150,-0.842090,-0.384225,-0.707361,0.594283,0.100501,0.064576,0.780991,-0.068667,4.447951,-2.103008,-0.356346,0.001572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,-0.148916,0.255536,0.050418,0.063653,-0.494292,-1.182638,-0.185208,-0.140737,-1.544378,-0.069825,-0.522023,0.592871,0.969871,0.106275
5317,-0.149857,0.215799,3.219931,-0.949145,-0.670456,0.700225,-0.185208,-0.148138,-0.198913,-0.069806,0.143840,-0.678933,1.255378,0.184321
5318,-0.150593,0.936079,-0.717003,0.952690,0.067164,1.518683,-0.185208,-0.019689,0.223056,-0.069697,-0.208876,-0.160995,0.756657,0.153632
5319,-0.148657,1.784303,-0.655317,-1.041496,0.039194,-0.601230,-0.185208,-0.230032,0.294809,-0.069054,-0.508312,0.550499,0.700999,0.165490


## Move data to gpu

In [3]:
device = gr.set_device()
print(f"Current device: {device.capitalize()}.")
X = np.array(df, dtype = np.float64)
X_gpu = torch.tensor(X, device = device, dtype = torch.float64)

Current device: Cuda.


# K is set as 10, which is based on the elbow method, and run wcss for single k function to get the labels

In [4]:
X, y, centroids, var = kmc.WCSS_for_single_k(X = X_gpu, k = 10, n_restarts = 50)
print(X.shape)
print(y.shape)
print(centroids.shape)
print(var)

Clustering with: k = 10.
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating cent

# SVC/OVO

In [5]:
import SVM.SVC as svc

Current training device: Cuda.


## Set parameters

In [6]:
epoch = int(1e6)
lr_cpu = 0.01
lr = torch.tensor(lr_cpu, device = X.device, dtype = X.dtype)
l2_panalty = True
print_every = int(100)
delta_loss_breaker = 1e-5
patience = int(10)

In [7]:
wb_dic = svc.ovo_train(
    X,
    y,
    num_epochs = epoch,
    start_learning_rate = lr,
    l2_penalty = l2_panalty,
    print_every = print_every,
    delta_loss_breaker = delta_loss_breaker,
    patience = patience
)

Creating random weights and bias with dtype: torch.float64
Training with loss function: hinge loss with l2 penalty on weights.
Epoch 0 | Loss: 1.2693151732767611 | Delta loss: None
Epoch 1 | Loss: 1.2529522125041141 | Delta loss: 0.016362960772646984
Epoch 100 | Loss: 0.028997044884811852 | Delta loss: 0.00032401481173590513
Epoch 200 | Loss: 0.017030429988847785 | Delta loss: 6.608300352359611e-05
Delta Loss smaller than threshold: Epoch 263 | Loss: 0.013389071471646356 | Delta loss: -3.478196122643634e-06
Delta Loss smaller than threshold: Epoch 273 | Loss: 0.012911800791051713 | Delta loss: 1.4900207623985695e-06
Delta Loss smaller than threshold: Epoch 285 | Loss: 0.012523583859291896 | Delta loss: 7.205930662773988e-06
Delta Loss smaller than threshold: Epoch 289 | Loss: 0.012343796520266199 | Delta loss: 6.486591587515078e-06
Delta Loss smaller than threshold: Epoch 295 | Loss: 0.012198995327204079 | Delta loss: -7.654587428343115e-06
Delta Loss smaller than threshold: Epoch 298 

In [9]:
keys = list(wb_dic.keys())
for key in keys:
    print(f"Key: {key}")
    print(f"weights and bias: {wb_dic[key]}")
    # print(f"Weights: {wb_dic[key][0]}")
    # print(f"Weights shape: {wb_dic[key][0].shape}")
    # print(f"Corresponding shape: {wb_dic[key].size()}")

Key: (0, 1)
weights and bias: tensor([ 0.7112, -2.2270,  0.4708,  0.3003,  0.2862,  0.5998,  0.4828,  0.1080,
         0.8192,  0.5537,  3.8192,  0.1111, -0.1122, -0.2176, -0.8086],
       device='cuda:0', dtype=torch.float64, grad_fn=<CatBackward0>)
Key: (0, 2)
weights and bias: tensor([-0.2491,  0.3442, -2.0521, -0.0726,  0.1414, -0.2147, -0.2249,  0.2464,
        -0.5719,  0.7904,  4.6320,  0.6067,  0.6034, -0.9761, -2.3578],
       device='cuda:0', dtype=torch.float64, grad_fn=<CatBackward0>)
Key: (0, 3)
weights and bias: tensor([-0.0147,  0.1461,  0.3670, -0.4080,  0.1526,  0.4220, -0.3298, -0.6691,
         0.3480,  0.9041,  0.6944, -1.1799, -0.2236,  0.3020,  1.1714],
       device='cuda:0', dtype=torch.float64, grad_fn=<CatBackward0>)
Key: (0, 4)
weights and bias: tensor([ 0.6006,  0.4882,  0.1948,  0.2179,  0.3164,  0.2364, -0.1881,  0.2266,
         0.5278,  0.1148,  1.5842,  0.0311,  0.0074,  0.3955,  0.5001],
       device='cuda:0', dtype=torch.float64, grad_fn=<CatBackward